In [ ]:
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
import os
import json
from dotenv import load_dotenv
from langchain_core.messages import SystemMessage, HumanMessage

load_dotenv(override=True)

In [ ]:
import subprocess
import socket

# Get Windows host IP from WSL
def get_windows_host_ip():
    # Method 1: Try WSL2 bridge interface (default route)
    try:
        result = subprocess.run(
            ["ip", "route", "show", "default"],
            capture_output=True,
            text=True
        )
        for line in result.stdout.split('\n'):
            if 'default' in line and 'via' in line:
                parts = line.split()
                via_index = parts.index('via')
                ip = parts[via_index + 1]
                # Test if Ollama is accessible at this IP
                try:
                    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    sock.settimeout(1)
                    sock.connect((ip, 11434))
                    sock.close()
                    print(f"✓ Found Ollama at {ip} (WSL default gateway)")
                    return ip
                except:
                    print(f"✗ Port 11434 not accessible at {ip}")
    except Exception as e:
        print(f"✗ Method 1 failed: {e}")
    
    # Method 2: Try /etc/resolv.conf nameserver
    try:
        result = subprocess.run(
            ["cat", "/etc/resolv.conf"],
            capture_output=True,
            text=True
        )
        for line in result.stdout.split('\n'):
            if 'nameserver' in line:
                ip = line.split()[1]
                # Test if Ollama is accessible at this IP
                try:
                    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    sock.settimeout(1)
                    sock.connect((ip, 11434))
                    sock.close()
                    print(f"✓ Found Ollama at {ip} (resolv.conf)")
                    return ip
                except:
                    print(f"✗ Port 11434 not accessible at {ip}")
    except Exception as e:
        print(f"✗ Method 2 failed: {e}")
    
    # Fallback to localhost
    print("⚠ Falling back to localhost")
    return "127.0.0.1"
    
WINDOWS_HOST_IP = get_windows_host_ip()

In [ ]:
filename_translation = {}

for folder in os.listdir("./assets/input"):
    for file in os.listdir(f"./assets/input/{folder}"):
        base_name = os.path.splitext(file)[0]
        filename_translation[base_name] = ""

with open(file="./filename_translation.json", mode="w", encoding="utf-8") as f:
    json.dump(filename_translation, f, ensure_ascii=False, indent=2)

In [ ]:
# llm = ChatOllama(base_url=f"http://{WINDOWS_HOST_IP}:11434",
#                  model="qwen3:1.7b",
#                  temperature=0,
#                  max_tokens=128)

GITHUB_TOKEN = os.environ.get("GITHUB_TOKEN")
llm = ChatOpenAI(
    model="openai/gpt-4.1-mini",
    temperature=0,
    base_url="https://models.github.ai/inference",
    api_key=GITHUB_TOKEN,
)

In [ ]:
def ch_to_eng(text: str) -> str:
    """Translate Chinese to English"""
    system = SystemMessage(
        content="You are a Mandarin to English translator. Only translate without adding or removing anything"
    )
    human = HumanMessage(content=text)
    resp = llm.invoke(input=[system, human])
    return resp.content

In [ ]:
for filename in list(filename_translation):
    if filename_translation[filename] == "":
        translated_filename = ch_to_eng(filename)
        filename_translation[filename] = translated_filename
        print(f"Translated {filename} to {translated_filename}")

        with open(file="./filename_translation.json", mode="w", encoding="utf-8") as f:
            json.dump(filename_translation, f, ensure_ascii=False, indent=2)